# **Introducción a R para Análisis de Datos**
## Capítulo 4: Limpieza de Datos 
---
Acerca de este notebook
* **Autor:** Juan Martin Bellido
* **Descripción:** *este notebook incluye las soluciones a los ejercicios del capítulo 4*
* **¿Feedback? ¿comentarios?** Por favor compártelo conmigo escribiéndome por [LinkedIn](https://www.linkedin.com/in/jmartinbellido/)


# Capítulo 4: Ejercicios
---

In [ ]:
# instalamos librería "data.table" porque Google colab no la incluye por defecto
install.packages("data.table")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘gridExtra’, ‘plyr’, ‘norm’, ‘visdat’, ‘viridis’, ‘UpSetR’




In [ ]:
# importamos las librerías que utilizaremos
require(dplyr)
require(data.table)
require(naniar)

Loading required package: naniar



In [ ]:
# (opcional) editamos las opciones globales para evitar que R utilice notación científica
options(scipen=999)

### Ejercicio #1

##### EX 1.A Identificar columnas con valores nulos (formalmente declarados) en el dataframe.
##### EX 1.B ¿Cuántas películas hay con valores nulos para el campo *director_name*?
##### EX 1.C Reemplazar valores nulos en *director_name* por "unknown".

> Dataset https://data-wizards.s3.amazonaws.com/datasets/movies.csv


In [ ]:
# importamos DataFrame
df_movies = fread("https://data-wizards.s3.amazonaws.com/datasets/movies.csv")
glimpse(df_movies)

Rows: 4,916
Columns: 28
$ color                     <chr> "Color", "Color", "Color", "Color", "", "Col…
$ director_name             <chr> "James Cameron", "Gore Verbinski", "Sam Mend…
$ num_critic_for_reviews    <int> 723, 302, 602, 813, NA, 462, 392, 324, 635, …
$ duration                  <int> 178, 169, 148, 164, NA, 132, 156, 100, 141, …
$ director_facebook_likes   <int> 0, 563, 0, 22000, 131, 475, 0, 15, 0, 282, 0…
$ actor_3_facebook_likes    <int> 855, 1000, 161, 23000, NA, 530, 4000, 284, 1…
$ actor_2_name              <chr> "Joel David Moore", "Orlando Bloom", "Rory K…
$ actor_1_facebook_likes    <int> 1000, 40000, 11000, 27000, 131, 640, 24000, …
$ gross                     <dbl> 760505847, 309404152, 200074175, 448130642, …
$ genres                    <chr> "Action|Adventure|Fantasy|Sci-Fi", "Action|A…
$ actor_1_name              <chr> "CCH Pounder", "Johnny Depp", "Christoph Wal…
$ movie_title               <chr> "Avatar", "Pirates of the Caribbean: At Worl…
$ num_voted_user

In [ ]:
# campos formalmente declarados como nulos
apply(is.na(df_movies),2,sum)

color             director_name    num_critic_for_reviews 
                        0                         0                        49 
                 duration   director_facebook_likes    actor_3_facebook_likes 
                       15                       102                        23 
             actor_2_name    actor_1_facebook_likes                     gross 
                        0                         7                       862 
                   genres              actor_1_name               movie_title 
                        0                         0                         0 
          num_voted_users cast_total_facebook_likes              actor_3_name 
                        0                         0                         0 
     facenumber_in_poster             plot_keywords           movie_imdb_link 
                       13                         0                         0 
     num_user_for_reviews                  language                   country 
                       21                         0                         0 
           content_rating                    budget                title_year 
                        0                       484                       106 
   actor_2_facebook_likes                imdb_score              aspect_ratio 
                       13                         0                       326 
     movie_facebook_likes 
                        0

In [ ]:
# para evaluar el campo "director_name", convertimos valores vacíos ("") en NA
df_movies$director_name = ifelse(df_movies$director_name == '', NA, df_movies$director_name)

In [ ]:
# cantidad de valores nulos en campo "director_name"
df_movies$director_name %>% is.na() %>% sum()

[1] 102

In [ ]:
# reemplazamos valores nulos por "unknown"
df_movies$director_name = ifelse(is.na(df_movies$director_name), 'unknown', df_movies$director_name)

In [ ]:
# volvemos a comprobar cantidad de valores nulos en campo "director_name"
df_movies$director_name %>% is.na() %>% sum()

[1] 0

### Ejercicio #2

Detectar empresas que sean outliers en cantidad de empleados.

##### EX 2.A Utilizando distribución normal
##### EX 2.B Utilizando el método del rango intercuartílico

> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv

In [ ]:
# importamos DataFrame
df_fortune = fread("https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv")
glimpse(df_fortune)

Rows: 1,000
Columns: 8
$ Rank      <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…
$ Company   <chr> "Walmart", "Exxon Mobil", "Apple", "Berkshire Hathaway", "Mc…
$ Sector    <chr> "Retailing", "Energy", "Technology", "Financials", "Health C…
$ Industry  <chr> "General Merchandisers", "Petroleum Refining", "Computers, O…
$ Location  <chr> "Bentonville, AR", "Irving, TX", "Cupertino, CA", "Omaha, NE…
$ Revenue   <int> 482130, 246204, 233715, 210821, 181241, 157107, 153290, 1523…
$ Profits   <int> 14694, 16150, 53394, 24083, 1476, 5813, 5237, 9687, 7373, 13…
$ Employees <int> 2300000, 75600, 110000, 331000, 70400, 200000, 199000, 21500…


Utilizando distribución normal.

In [ ]:
# calculamos la media y la desviación estándar
mean <- mean(df_fortune$Employees)
sd <- sd(df_fortune$Employees)

In [ ]:
# convertimos campo a z y lo almacenamos en una nueva columna
df_fortune$z_employees = (df_fortune$Employees - mean) / sd

In [ ]:
# visualizarmos outliers
df_fortune %>% filter(
  abs(z_employees) > 3 
)

Rank,Company,Sector,Industry,Location,Revenue,Profits,Employees,z_employees
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>
1,Walmart,Retailing,General Merchandisers,"Bentonville, AR",482130,14694,2300000,25.069445
4,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",210821,24083,331000,3.295141
11,General Electric,Industrials,Industrial Machinery,"Fairfield, CT",140389,-6126,333000,3.317258
17,Kroger,Food and Drug Stores,Food and Drug Stores,"Cincinnati, OH",109830,2039,431000,4.400997
28,Home Depot,Retailing,Specialty Retailers: Other,"Atlanta, GA",88519,7009,385000,3.892303
31,IBM,Technology,Information Technology Services,"Armonk, NY",82461,13190,411798,4.188651
38,Target,Retailing,General Merchandisers,"Minneapolis, MN",73785,3363,341000,3.405727
48,UPS,Transportation,"Mail, Package, and Freight Delivery","Atlanta, GA",58363,4844,341240,3.408381
58,FedEx,Transportation,"Mail, Package, and Freight Delivery","Memphis, TN",47453,1050,323035,3.207060


Utilizando método del rango intercuartílico.

In [ ]:
# calculamos el rango intercuartilico (IQR) como la diferencia entre el percentil 75 y el 25
IQR <- quantile(df_fortune$Employees, 0.75) - quantile(df_fortune$Employees, 0.25)

In [ ]:
# calculamos los límites inferior y superior
lower_limit <- quantile(df_fortune$Employees, 0.25) - 1.5 * IQR
upper_limit <- quantile(df_fortune$Employees, 0.75) + 1.5 * IQR

In [ ]:
# filtramos observaciones que se encuentren por fuera de estos límites
df_fortune %>% filter(
  Employees < lower_limit | Employees > upper_limit
)

Rank,Company,Sector,Industry,Location,Revenue,Profits,Employees,z_employees
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>
1,Walmart,Retailing,General Merchandisers,"Bentonville, AR",482130,14694,2300000,25.0694448
2,Exxon Mobil,Energy,Petroleum Refining,"Irving, TX",246204,16150,75600,0.4707850
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000,0.8511994
4,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",210821,24083,331000,3.2951411
5,McKesson,Health Care,Wholesalers: Health Care,"San Francisco, CA",181241,1476,70400,0.4132805
6,UnitedHealth Group,Health Care,Health Care: Insurance and Managed Care,"Minnetonka, MN",157107,5813,200000,1.8464698
7,CVS Health,Food and Drug Stores,Food and Drug Stores,"Woonsocket, RI",153290,5237,199000,1.8354112
8,General Motors,Motor Vehicles & Parts,Motor Vehicles and Parts,"Detroit, MI",152356,9687,215000,2.0123482
9,Ford Motor,Motor Vehicles & Parts,Motor Vehicles and Parts,"Dearborn, MI",149558,7373,199000,1.8354112
